In [1]:
import numpy as np

# 假设有两个数组
array1 = np.array([[1, 2, 3], [4, 5, 6]])  # 形状为 (2, 3)
array2 = np.array([[7, 8, 9], [10, 11, 12]])  # 形状为 (2, 3)

# 在每个样本上添加一个维度
array1_expanded = np.expand_dims(array1, axis=2)  # 形状变为 (2, 3, 1)
array2_expanded = np.expand_dims(array2, axis=2)  # 形状变为 (2, 3, 1)

# 垂直拼接两个数组
result = np.concatenate((array1_expanded, array2_expanded), axis=0)

# 输出结果
print(result.shape)  # 打印结果的形状
print(result)

背包中能装入的物品的最大总价值为: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 160, 160, 160, 160, 160, 160, 160, 160, 160, 160, 180, 180, 180, 180, 180, 180, 180, 180, 180, 180, 220]]
